# Finite Differences CapSol Example

Minimal example of using capsol python code to compute the voltage between a SPM tip (sphere, cone, and cantilever body disk) and a metal surface.

In [1]:
 %load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [11]:

import numpy as np
from scipy import linalg
from scipy import sparse
from scipy.sparse import linalg as la
import capsol
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [12]:
params= capsol.Params(Nr=900, Nz_plus=900, rhoMax=20*20000.0, zMax=20*20000.0)

In [13]:
sim = capsol.CapSol(params)

In [14]:
params

Params(Rtip=20.0, theta_deg=15.0, Hcone=15000.0, Hcant=500.0, Rcant=15000.0, zMax=400000.0, rhoMax=400000.0, h0=0.4, d=20.0, Nuni=50, Nr=900, Nz_plus=900, hsam=0.0, theta=0.2617993877991494)

In [29]:
def Totalsim(params, dmin, dmax, istep, fname):
    capacitances=[]
    distances=np.arange(dmin, dmax, istep*params.h0)
    for d in tqdm(distances):
        params.d= d
        sim = capsol.CapSol(params)
        sim.setup_matrices()
        sim.solve()
        sim.process()
        capacitances.append(sim.c)
    np.savetxt(fname, np.c_[distances, capacitances], header='distance (nm) Capacitances(F)', footer=f'Totalsim(params={params}, dmin={dmin}, dmax={dmax}, istep={istep}, fname={fname})')
    return distances, capacitances

In [30]:
dmin=10
dmax=20
istep=4

In [32]:
Totalsim(params, dmin, dmax, istep, 'D-C.dat')

  0%|          | 0/7 [00:00<?, ?it/s]

(array([10. , 11.6, 13.2, 14.8, 16.4, 18. , 19.6]),
 [3.417254055824126e-15,
  3.4166697736241165e-15,
  3.416123749962859e-15,
  3.415607438975718e-15,
  3.4151149343880347e-15,
  3.4146419831005703e-15,
  3.414185419702169e-15])

In [18]:
capacitances=[]
for d in tqdm(distances):
    params.d= d
    sim = capsol.CapSol(params)
    sim.setup_matrices()
    sim.solve()
    sim.process()
    capacitances.append(sim.c)

  0%|          | 0/7 [00:00<?, ?it/s]

In [10]:
from tqdm.notebook import tqdm

In [ ]:
plt.imshow(sim.spm_tip[::-1, :])


In [19]:
capacitances

[3.417254055824126e-15,
 3.4166697736241165e-15,
 3.416123749962859e-15,
 3.415607438975718e-15,
 3.4151149343880347e-15,
 3.4146419831005703e-15,
 3.414185419702169e-15]

In [27]:
np.savetxt(r'C:\Users\emaro\OneDrive - University of Mount Union\XSEDE Summer 2021\xsede-spm\capsol\D-C.dat', np.c_[distances, capacitances], header='distance (nm) Capacitances(F)', footer='', )

In [23]:
cd

C:\Users\emaro


In [ ]:
full_boundary = sim.boundary
A_cut = sim.A[~full_boundary].T[~full_boundary].T # 
f_cut = f[~full_boundary]

In [ ]:
1/A_cut.diagonal()

In [ ]:
%time u_cut = la.spsolve(A_cut, f_cut)

In [ ]:
u = sim.u.ravel()
u[~full_boundary] = u_cut

In [ ]:
fig, ax = plt.subplots()
s = np.s_[:, :]
plt.contourf(sim.R[s], sim.Z[s], u.reshape((sim.Nz, sim.Nr))[s], levels=20)
plt.colorbar()
ax.set_aspect('equal')

In [ ]:
dV = capsol.grid_area(sim.r, sim.z)

In [ ]:
energy = 0.5 * np.sum(dV * abs(capsol.E_field(u, sim.r, sim.z))**2) * 1e-9 * 8.854e-12

In [ ]:
energy

In [ ]:
energy_z = 0.5 * np.sum(dV * capsol.E_field(u, sim.r, sim.z).imag**2) * 1e-9 * 8.854e-12

In [ ]:
energy_z

In [ ]:
# All the energy related to the radial field is not helpful / useful...

In [ ]:
E_J = energy * 1e-9 * 8.854e-12
E_J

In [ ]:
sim.energy

In [ ]:
# E-field is V/m, here V/nm...
# dV = nm^3
# Units are V^2 * nm...